In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [25]:
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_ri.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,777541,9179517702,5/1/20,165,PPP,"COASTAL MEDICAL, INC.",10 Davol Square Suite 300,Providnce,RI,2903.0,...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,500 or more,"10 Davol Square Suite 300, Providnce, RI",41.804245,-71.420240,4.400700e+14
1,777542,9713307002,4/9/20,165,PPP,BROWN MEDICINE,110 ELM ST,PROVIDENCE,RI,2903.0,...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"110 ELM ST, PROVIDENCE, RI",41.817000,-71.410620,4.400700e+14
2,777543,5711067207,4/27/20,165,PPP,TACO INC,1160 CRANSTON ST,CRANSTON,RI,2920.0,...,333 Machinery,"3334 Ventilation, Heating, Air-Conditioning, a...",3334.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1160 CRANSTON ST, CRANSTON, RI",41.795063,-71.449750,4.400701e+14
3,777544,4819187009,4/4/20,165,PPP,"UNIVERSITY ORTHOPEDICS, INC.",1 Kettle Point Avenue,EAST PROVIDENCE,RI,2914.0,...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1 Kettle Point Avenue, EAST PROVIDENCE, RI",41.796700,-71.375510,4.400701e+14
4,777545,3895377102,4/12/20,165,PPP,"H. CARR & SONS, INC.",100 Royal Little Drive,PROVIDENCE,RI,2904.0,...,999 Other,2383 Building Finishing Contractors,2383.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"100 Royal Little Drive, PROVIDENCE, RI",41.847070,-71.413536,4.400700e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3787,781328,8379078305,1/29/21,165,PPS,GRAPHIC INNOVATIONS INC,380 Jefferson Blvd Ste C,Warwick,RI,2886.0,...,323 Printing & Publishing,3231 Printing and Related Support Activities,3231.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"380 Jefferson Blvd Ste C, Warwick, RI",41.738247,-71.439550,4.400302e+14
3788,781329,8836008607,3/25/21,165,PPS,NEW ENGLAND DRYWALL CO.,816 Middle Rd Unit 5,East Greenwich,RI,2818.0,...,999 Other,2383 Building Finishing Contractors,2383.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"816 Middle Rd Unit 5, East Greenwich, RI",41.645916,-71.489230,4.400302e+14
3789,781330,8960777006,4/9/20,165,PPP,WATER DAMAGE DRYING SERVICES INC.,12 GAVITT AVE,WESTERLY,RI,2891.0,...,999 Other,2389 Other Specialty Trade Contractors,2389.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"12 GAVITT AVE, WESTERLY, RI",41.383280,-71.820860,4.400905e+14
3790,781331,9102547103,4/15/20,165,PPP,STORMTITE COMPANY INC.,1065 WARWICK AVE,WARWICK,RI,2888.0,...,999 Other,2361 Residential Building Construction,2361.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1065 WARWICK AVE, WARWICK, RI",41.746986,-71.405410,4.400302e+14


In [26]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [27]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long


In [28]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,9179517702,"10 Davol Square Suite 300, Providnce, RI",41.804245,-71.420240
1,9713307002,"110 ELM ST, PROVIDENCE, RI",41.817000,-71.410620
2,5711067207,"1160 CRANSTON ST, CRANSTON, RI",41.795063,-71.449750
3,4819187009,"1 Kettle Point Avenue, EAST PROVIDENCE, RI",41.796700,-71.375510
4,3895377102,"100 Royal Little Drive, PROVIDENCE, RI",41.847070,-71.413536


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [29]:
fips_check_scraped = pd.read_csv("../../data/state_data/geo/geo_fips/RI_fips_scraped.csv") 
fips_check_scraped.head() #15869

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,777541,777541,777541,9179517702,5/1/20,165,PPP,"COASTAL MEDICAL, INC.",10 Davol Square Suite 300,...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,500 or more,"10 Davol Square Suite 300, Providnce, RI",41.804245,-71.420240,4.400700e+14
1,1,777542,777542,777542,9713307002,4/9/20,165,PPP,BROWN MEDICINE,110 ELM ST,...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"110 ELM ST, PROVIDENCE, RI",41.817000,-71.410620,4.400700e+14
2,2,777543,777543,777543,5711067207,4/27/20,165,PPP,TACO INC,1160 CRANSTON ST,...,333 Machinery,"3334 Ventilation, Heating, Air-Conditioning, a...",3334.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1160 CRANSTON ST, CRANSTON, RI",41.795063,-71.449750,4.400701e+14
3,3,777544,777544,777544,4819187009,4/4/20,165,PPP,"UNIVERSITY ORTHOPEDICS, INC.",1 Kettle Point Avenue,...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1 Kettle Point Avenue, EAST PROVIDENCE, RI",41.796700,-71.375510,4.400701e+14
4,4,777545,777545,777545,3895377102,4/12/20,165,PPP,"H. CARR & SONS, INC.",100 Royal Little Drive,...,999 Other,2383 Building Finishing Contractors,2383.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"100 Royal Little Drive, PROVIDENCE, RI",41.847070,-71.413536,4.400700e+14


In [30]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [31]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [32]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,9179517702,"10 Davol Square Suite 300, Providnce, RI",41.804245,-71.420240,4.400700e+14
1,9713307002,"110 ELM ST, PROVIDENCE, RI",41.817000,-71.410620,4.400700e+14
2,5711067207,"1160 CRANSTON ST, CRANSTON, RI",41.795063,-71.449750,4.400701e+14
3,4819187009,"1 Kettle Point Avenue, EAST PROVIDENCE, RI",41.796700,-71.375510,4.400701e+14
4,3895377102,"100 Royal Little Drive, PROVIDENCE, RI",41.847070,-71.413536,4.400700e+14
...,...,...,...,...,...
3787,8379078305,"380 Jefferson Blvd Ste C, Warwick, RI",41.738247,-71.439550,4.400302e+14
3788,8836008607,"816 Middle Rd Unit 5, East Greenwich, RI",41.645916,-71.489230,4.400302e+14
3789,8960777006,"12 GAVITT AVE, WESTERLY, RI",41.383280,-71.820860,4.400905e+14
3790,9102547103,"1065 WARWICK AVE, WARWICK, RI",41.746986,-71.405410,4.400302e+14


In [33]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #5503

,LoanNumber,full_add,Lat,Long,FIPS_z
3310,1620187106,"2700 PLAINFIELD PIKE, CRANSTON, RI",41.789741,-71.538455,NaN
3317,7313537003,"567 SOUTH COUNTY TRAIL SUITE 107, EXETER, RI",41.527457,-71.552202,NaN
3334,3649398708,"541 HARTFORD AVE, JOHNSTON, RI",41.819151,-71.461754,NaN
3335,8830027006,"P.O. Box 132, LITTLE COMPTON, RI",41.510103,-71.171156,NaN
3355,4574717003,"15 Patrol Road, NORTH KINGSTOWN, RI",41.619406,-71.409574,NaN
3363,6111687108,"1485 South County Trail Unit 302 South, EAST G...",41.656178,-71.498483,NaN
3365,1327547206,"16 TERRY LANE, UNIT 3A, Glocester, RI",41.903695,-71.646085,NaN
3373,6888298409,"147 Industrial Dr, North Smithfield, RI",41.990678,-71.563354,NaN
3374,1289998609,"300 Wampanoag Trl, Riverside, RI",41.796317,-71.350428,NaN
3375,8125607106,"300 Wampanaog Trail, East Providence, RI",41.796317,-71.350428,NaN


<h4>FIPS Script

In [34]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [35]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
#driver = webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))
#executable_path = {'executable_path': webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))}
executable_path = {'executable_path': ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()}
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', **executable_path, options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('44')] #RI
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/56 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [36]:
geo_id_list =geo_id 

Length Check

In [37]:
print(len(geo_id_list))

56


Matching FIPS to DataFrame

In [38]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:3308] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null

In [39]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LoanNumber,full_add,Lat,Long,FIPS_z
3310,1620187106,"2700 PLAINFIELD PIKE, CRANSTON, RI",41.789741,-71.538455,[440070146002006]
3317,7313537003,"567 SOUTH COUNTY TRAIL SUITE 107, EXETER, RI",41.527457,-71.552202,[440090505001036]
3334,3649398708,"541 HARTFORD AVE, JOHNSTON, RI",41.819151,-71.461754,[440070018001017]
3335,8830027006,"P.O. Box 132, LITTLE COMPTON, RI",41.510103,-71.171156,[440050414003021]
3355,4574717003,"15 Patrol Road, NORTH KINGSTOWN, RI",41.619406,-71.409574,[440090501032004]
3363,6111687108,"1485 South County Trail Unit 302 South, EAST G...",41.656178,-71.498483,[440030209031023]
3365,1327547206,"16 TERRY LANE, UNIT 3A, Glocester, RI",41.903695,-71.646085,[440070131021004]
3373,6888298409,"147 Industrial Dr, North Smithfield, RI",41.990678,-71.563354,[440070128031034]
3374,1289998609,"300 Wampanoag Trl, Riverside, RI",41.796317,-71.350428,[440070105011032]
3375,8125607106,"300 Wampanaog Trail, East Providence, RI",41.796317,-71.350428,[440070105011032]


Remove brackets

In [40]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LoanNumber,full_add,Lat,Long,FIPS_z
3310,1620187106,"2700 PLAINFIELD PIKE, CRANSTON, RI",41.789741,-71.538455,440070146002006
3317,7313537003,"567 SOUTH COUNTY TRAIL SUITE 107, EXETER, RI",41.527457,-71.552202,440090505001036
3334,3649398708,"541 HARTFORD AVE, JOHNSTON, RI",41.819151,-71.461754,440070018001017
3335,8830027006,"P.O. Box 132, LITTLE COMPTON, RI",41.510103,-71.171156,440050414003021
3355,4574717003,"15 Patrol Road, NORTH KINGSTOWN, RI",41.619406,-71.409574,440090501032004
3363,6111687108,"1485 South County Trail Unit 302 South, EAST G...",41.656178,-71.498483,440030209031023
3365,1327547206,"16 TERRY LANE, UNIT 3A, Glocester, RI",41.903695,-71.646085,440070131021004
3373,6888298409,"147 Industrial Dr, North Smithfield, RI",41.990678,-71.563354,440070128031034
3374,1289998609,"300 Wampanoag Trl, Riverside, RI",41.796317,-71.350428,440070105011032
3375,8125607106,"300 Wampanaog Trail, East Providence, RI",41.796317,-71.350428,440070105011032


In [41]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z


Parse for Merge

In [42]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
3310,1620187106,440070146002006
3317,7313537003,440090505001036
3334,3649398708,440070018001017
3335,8830027006,440050414003021
3355,4574717003,440090501032004
3363,6111687108,440030209031023
3365,1327547206,440070131021004
3373,6888298409,440070128031034
3374,1289998609,440070105011032
3375,8125607106,440070105011032


Read in base data

In [43]:
#if from geocode file
#fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_ri.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
#fips_check.head()

#if from FIPS file
fips_check = pd.read_csv("../../data/state_data/geo/geo_fips/RI_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head() #22826


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,777541,777541,777541,9179517702,5/1/20,165,PPP,"COASTAL MEDICAL, INC.",10 Davol Square Suite 300,...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,500 or more,"10 Davol Square Suite 300, Providnce, RI",41.804245,-71.420240,4.400700e+14
1,1,777542,777542,777542,9713307002,4/9/20,165,PPP,BROWN MEDICINE,110 ELM ST,...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"110 ELM ST, PROVIDENCE, RI",41.817000,-71.410620,4.400700e+14
2,2,777543,777543,777543,5711067207,4/27/20,165,PPP,TACO INC,1160 CRANSTON ST,...,333 Machinery,"3334 Ventilation, Heating, Air-Conditioning, a...",3334.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1160 CRANSTON ST, CRANSTON, RI",41.795063,-71.449750,4.400701e+14
3,3,777544,777544,777544,4819187009,4/4/20,165,PPP,"UNIVERSITY ORTHOPEDICS, INC.",1 Kettle Point Avenue,...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1 Kettle Point Avenue, EAST PROVIDENCE, RI",41.796700,-71.375510,4.400701e+14
4,4,777545,777545,777545,3895377102,4/12/20,165,PPP,"H. CARR & SONS, INC.",100 Royal Little Drive,...,999 Other,2383 Building Finishing Contractors,2383.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"100 Royal Little Drive, PROVIDENCE, RI",41.847070,-71.413536,4.400700e+14


Merge with MainFrame

In [44]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,777541,777541,777541,9179517702,5/1/20,165,PPP,"COASTAL MEDICAL, INC.",10 Davol Square Suite 300,...,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,500 or more,"10 Davol Square Suite 300, Providnce, RI",41.804245,-71.420240,4.400700e+14,NaN
1,1,777542,777542,777542,9713307002,4/9/20,165,PPP,BROWN MEDICINE,110 ELM ST,...,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"110 ELM ST, PROVIDENCE, RI",41.817000,-71.410620,4.400700e+14,NaN
2,2,777543,777543,777543,5711067207,4/27/20,165,PPP,TACO INC,1160 CRANSTON ST,...,"3334 Ventilation, Heating, Air-Conditioning, a...",3334.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1160 CRANSTON ST, CRANSTON, RI",41.795063,-71.449750,4.400701e+14,NaN
3,3,777544,777544,777544,4819187009,4/4/20,165,PPP,"UNIVERSITY ORTHOPEDICS, INC.",1 Kettle Point Avenue,...,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1 Kettle Point Avenue, EAST PROVIDENCE, RI",41.796700,-71.375510,4.400701e+14,NaN
4,4,777545,777545,777545,3895377102,4/12/20,165,PPP,"H. CARR & SONS, INC.",100 Royal Little Drive,...,2383 Building Finishing Contractors,2383.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"100 Royal Little Drive, PROVIDENCE, RI",41.847070,-71.413536,4.400700e+14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3787,3787,781328,781328,781328,8379078305,1/29/21,165,PPS,GRAPHIC INNOVATIONS INC,380 Jefferson Blvd Ste C,...,3231 Printing and Related Support Activities,3231.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"380 Jefferson Blvd Ste C, Warwick, RI",41.738247,-71.439550,4.400302e+14,NaN
3788,3788,781329,781329,781329,8836008607,3/25/21,165,PPS,NEW ENGLAND DRYWALL CO.,816 Middle Rd Unit 5,...,2383 Building Finishing Contractors,2383.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"816 Middle Rd Unit 5, East Greenwich, RI",41.645916,-71.489230,4.400302e+14,NaN
3789,3789,781330,781330,781330,8960777006,4/9/20,165,PPP,WATER DAMAGE DRYING SERVICES INC.,12 GAVITT AVE,...,2389 Other Specialty Trade Contractors,2389.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"12 GAVITT AVE, WESTERLY, RI",41.383280,-71.820860,4.400905e+14,NaN
3790,3790,781331,781331,781331,9102547103,4/15/20,165,PPP,STORMTITE COMPANY INC.,1065 WARWICK AVE,...,2361 Residential Building Construction,2361.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1065 WARWICK AVE, WARWICK, RI",41.746986,-71.405410,4.400302e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [45]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,777541,777541,777541,9179517702,5/1/20,165,PPP,"COASTAL MEDICAL, INC.",10 Davol Square Suite 300,...,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,500 or more,"10 Davol Square Suite 300, Providnce, RI",41.804245,-71.420240,4.400700e+14,NaN,440070003022004.0
1,1,777542,777542,777542,9713307002,4/9/20,165,PPP,BROWN MEDICINE,110 ELM ST,...,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"110 ELM ST, PROVIDENCE, RI",41.817000,-71.410620,4.400700e+14,NaN,440070006001018.0
2,2,777543,777543,777543,5711067207,4/27/20,165,PPP,TACO INC,1160 CRANSTON ST,...,3334.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1160 CRANSTON ST, CRANSTON, RI",41.795063,-71.449750,4.400701e+14,NaN,440070141003052.0
3,3,777544,777544,777544,4819187009,4/4/20,165,PPP,"UNIVERSITY ORTHOPEDICS, INC.",1 Kettle Point Avenue,...,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1 Kettle Point Avenue, EAST PROVIDENCE, RI",41.796700,-71.375510,4.400701e+14,NaN,440070105021015.0
4,4,777545,777545,777545,3895377102,4/12/20,165,PPP,"H. CARR & SONS, INC.",100 Royal Little Drive,...,2383.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"100 Royal Little Drive, PROVIDENCE, RI",41.847070,-71.413536,4.400700e+14,NaN,440070029004015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3787,3787,781328,781328,781328,8379078305,1/29/21,165,PPS,GRAPHIC INNOVATIONS INC,380 Jefferson Blvd Ste C,...,3231.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"380 Jefferson Blvd Ste C, Warwick, RI",41.738247,-71.439550,4.400302e+14,NaN,440030211004009.0
3788,3788,781329,781329,781329,8836008607,3/25/21,165,PPS,NEW ENGLAND DRYWALL CO.,816 Middle Rd Unit 5,...,2383.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"816 Middle Rd Unit 5, East Greenwich, RI",41.645916,-71.489230,4.400302e+14,NaN,440030209032001.0
3789,3789,781330,781330,781330,8960777006,4/9/20,165,PPP,WATER DAMAGE DRYING SERVICES INC.,12 GAVITT AVE,...,2389.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"12 GAVITT AVE, WESTERLY, RI",41.383280,-71.820860,4.400905e+14,NaN,440090508012001.0
3790,3790,781331,781331,781331,9102547103,4/15/20,165,PPP,STORMTITE COMPANY INC.,1065 WARWICK AVE,...,2361.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1065 WARWICK AVE, WARWICK, RI",41.746986,-71.405410,4.400302e+14,NaN,440030212002020.0


Drop Extra FIPS Columns

In [46]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,777541,777541,777541,9179517702,5/1/20,165,PPP,"COASTAL MEDICAL, INC.",10 Davol Square Suite 300,...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,500 or more,"10 Davol Square Suite 300, Providnce, RI",41.804245,-71.420240,440070003022004.0
1,1,777542,777542,777542,9713307002,4/9/20,165,PPP,BROWN MEDICINE,110 ELM ST,...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"110 ELM ST, PROVIDENCE, RI",41.817000,-71.410620,440070006001018.0
2,2,777543,777543,777543,5711067207,4/27/20,165,PPP,TACO INC,1160 CRANSTON ST,...,333 Machinery,"3334 Ventilation, Heating, Air-Conditioning, a...",3334.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1160 CRANSTON ST, CRANSTON, RI",41.795063,-71.449750,440070141003052.0
3,3,777544,777544,777544,4819187009,4/4/20,165,PPP,"UNIVERSITY ORTHOPEDICS, INC.",1 Kettle Point Avenue,...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1 Kettle Point Avenue, EAST PROVIDENCE, RI",41.796700,-71.375510,440070105021015.0
4,4,777545,777545,777545,3895377102,4/12/20,165,PPP,"H. CARR & SONS, INC.",100 Royal Little Drive,...,999 Other,2383 Building Finishing Contractors,2383.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"100 Royal Little Drive, PROVIDENCE, RI",41.847070,-71.413536,440070029004015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3787,3787,781328,781328,781328,8379078305,1/29/21,165,PPS,GRAPHIC INNOVATIONS INC,380 Jefferson Blvd Ste C,...,323 Printing & Publishing,3231 Printing and Related Support Activities,3231.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"380 Jefferson Blvd Ste C, Warwick, RI",41.738247,-71.439550,440030211004009.0
3788,3788,781329,781329,781329,8836008607,3/25/21,165,PPS,NEW ENGLAND DRYWALL CO.,816 Middle Rd Unit 5,...,999 Other,2383 Building Finishing Contractors,2383.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"816 Middle Rd Unit 5, East Greenwich, RI",41.645916,-71.489230,440030209032001.0
3789,3789,781330,781330,781330,8960777006,4/9/20,165,PPP,WATER DAMAGE DRYING SERVICES INC.,12 GAVITT AVE,...,999 Other,2389 Other Specialty Trade Contractors,2389.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"12 GAVITT AVE, WESTERLY, RI",41.383280,-71.820860,440090508012001.0
3790,3790,781331,781331,781331,9102547103,4/15/20,165,PPP,STORMTITE COMPANY INC.,1065 WARWICK AVE,...,999 Other,2361 Residential Building Construction,2361.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1065 WARWICK AVE, WARWICK, RI",41.746986,-71.405410,440030212002020.0


In [47]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z


Convert to File

In [48]:
fips_ri = fips_merge_drop

In [49]:
fips_ri.to_csv("../../data/state_data/geo/geo_fips/RI_fips_scraped.csv") 

Review & Compare

In [50]:
geo_ri_fips = pd.read_csv("../../data/state_data/geo/geo_fips/RI_fips_scraped.csv") 
geo_ri_fips.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,777541,777541,777541,9179517702,5/1/20,165,PPP,"COASTAL MEDICAL, INC.",...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,500 or more,"10 Davol Square Suite 300, Providnce, RI",41.804245,-71.420240,4.400700e+14
1,1,1,777542,777542,777542,9713307002,4/9/20,165,PPP,BROWN MEDICINE,...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"110 ELM ST, PROVIDENCE, RI",41.817000,-71.410620,4.400700e+14
2,2,2,777543,777543,777543,5711067207,4/27/20,165,PPP,TACO INC,...,333 Machinery,"3334 Ventilation, Heating, Air-Conditioning, a...",3334.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1160 CRANSTON ST, CRANSTON, RI",41.795063,-71.449750,4.400701e+14
3,3,3,777544,777544,777544,4819187009,4/4/20,165,PPP,"UNIVERSITY ORTHOPEDICS, INC.",...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1 Kettle Point Avenue, EAST PROVIDENCE, RI",41.796700,-71.375510,4.400701e+14
4,4,4,777545,777545,777545,3895377102,4/12/20,165,PPP,"H. CARR & SONS, INC.",...,999 Other,2383 Building Finishing Contractors,2383.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"100 Royal Little Drive, PROVIDENCE, RI",41.847070,-71.413536,4.400700e+14


In [51]:
##### fill in missing Lat, Long, FIPS
geo_ri_fips.loc[geo_ri_fips["FIPS_z"].isnull()] #27

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
